In [1]:
## SETUP 
imports = ['wrds', 'pandas as pd', 'os','math', 'glob',
           're', 'pickle', 'numpy as np',
           'from name_matching.name_matcher import NameMatcher',
          'from joblib import Parallel, delayed',
          'from IPython.display import display, HTML, clear_output',
          'unicodedata', 'sys', 'numpy as np', 'glob','from sklearn.decomposition import PCA']
for command in imports:
    if command.startswith('from'): exec(command)
    else: exec('import ' + command)

if not os.getcwd().endswith('Big Data'):
    os.chdir('../..')

sys.path.append('trade_data_code/2_python')
processed_linkedin = '1) data/15_revelio_outputs/1_inputs/b_processed_data/linkedin/'
processed_admin = '1) data/15_revelio_outputs/1_inputs/b_processed_data/admin/'
import A_helper_functions as hf


In [ ]:
########################################################################################
# Perform PCA to generate metrics of differentiation 
########################################################################################
temp_direct = processed_linkedin + 'temp_role'
os.makedirs(temp_direct, exist_ok=True)

def collapse_year_level(year, making_pca, weight_var, pca_model=None):
    temp = (
        output.assign(
            valid=lambda x: x['startdate'].dt.year.le(year) & 
                            (x['enddate'].isna() | x['enddate'].dt.year.ge(year)),
            wgted_comp=lambda x: x['total_compensation'] * x['weight']
        )
        .loc[lambda x: x['valid']]
        .groupby(['firmid', 'role_k1500'], as_index=False)
        .agg(comp=(weight_var, 'sum'))
        .assign(year=year)
        .pivot_table(index=['firmid', 'year'], columns='role_k1500', values='comp', aggfunc='sum', fill_value=0)
        .pipe(lambda df: df.div(df.sum(axis=1), axis=0))
        .replace([np.inf, -np.inf], np.nan)
        .dropna()
    )
    if making_pca:
        pca_model = PCA(n_components=10)
        pca_model.fit(temp)
        return pca_model
    else:
        file_path = temp_direct + "/temp" + str(year) + ".parquet"
        pd.concat([
            temp.reset_index()[['firmid', 'year']],
            pd.DataFrame(pca_model.transform(temp), columns=[f'{weight_var}_PC{i+1}' for i in range(10)])
        ], axis=1).to_parquet(file_path)
        print(year)
        
#set param values 
years = range(2008, 2024)
sample_year = 2015
   
# Load and merge data
long_data = pd.read_parquet(processed_linkedin + 'matched_firm_role_output.parquet')
matching_output = pd.read_parquet(processed_admin +'fuzzy_matching_output_final.parquet')[['rcid', 'siren']].rename(columns={'siren': 'firmid'})
output = pd.merge(long_data, matching_output)

In [ ]:
# run pca analysis
for weight_var in ['weight']:
    print('starting pca gen')
    pca_model = collapse_year_level(sample_year, True, weight_var)
    print('finished pca gen')
    [collapse_year_level(year,False,weight_var,pca_model) for year in range(2018, 2024)]
    (pd.concat([pd.read_parquet(file) for file in glob.glob(temp_direct + "/*.parquet")],ignore_index = True)
     .to_parquet(processed_linkedin + 'matched_firm_pca_'+ weight_var + '_output.parquet'))
shutil.rmtree(temp_direct)

In [ ]:
weight_var = 'weight'
pca_model = collapse_year_level(sample_year, True, weight_var)

In [ ]:
year = 2018
temp = (
    output.assign(
        valid=lambda x: x['startdate'].dt.year.le(year) & 
                        (x['enddate'].isna() | x['enddate'].dt.year.ge(year)),
        wgted_comp=lambda x: x['total_compensation'] * x['weight']
    )
    .loc[lambda x: x['valid']]
    .groupby(['firmid', 'role_k1500'], as_index=False)
    .agg(comp=(weight_var, 'sum'))
    .assign(year=year)
    .pivot_table(index=['firmid', 'year'], columns='role_k1500', values='comp', aggfunc='sum', fill_value=0)
    .pipe(lambda df: df.div(df.sum(axis=1), axis=0))
    .replace([np.inf, -np.inf], np.nan)
    .dropna()
)

In [ ]:
weight_var = 'weight'